# Analysis of NGC 3627 with Optical + X-ray wavelength data

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astroquery.mast import Observations

from astropy.io import fits
from astropy.stats import sigma_clipped_stats, mad_std

from photutils.detection import DAOStarFinder

import shutil
import glob
import os 

cd = os.chdir
hstdir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/HST/"

# Mosaicking
## Querying data with astroquery

In [7]:
filters = ["F275W", "F336W", "F438W", "F555W", "F814W"]
obs_id = [13364, 13477, 15654, 12968]

print("Finding observations for WFC3/UVIS images...")
obs_table = Observations.query_criteria(objectname="M66", obs_collection="HST",
                                        filters=filters,
                                        proposal_id=obs_id, instrument_name="WFC3/UVIS")
print("Done")

Finding observations for WFC3/UVIS images...
Done


In [8]:
obs_table

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID,objID1,distance
str7,str3,str7,str9,str3,str5,str7,str9,str41,str35,float64,float64,str5,str20,int64,float64,float64,float64,float64,float64,str90,float64,str5,str3,int64,str912,str60,str61,str6,bool,float64,str8,str9,str9,float64
science,HST,CALWF3,WFC3/UVIS,HST,F275W,UV,NGC-3627,GALAXY;SPIRAL,idxr22060,170.0646666667,13.00013888889,image,"Lee, Janice",3,58798.6967849537,58798.854620335645,2190.0,243.20000000000002,305.8,PHANGS-HST: Linking Stars and Gas throughout the Scales of Star Formation,58799.06275462,15654,GO,--,POLYGON 170.08698664 13.00556577 170.07985413 13.02716006 170.07982858651627 13.027153744369755 170.0796716 13.02762899 170.07964579535579 13.027622609763295 170.07948883 13.02809779 170.0343843 13.01694125 170.04152047 12.99534819 170.0415462823843 12.995354576975101 170.04170331 12.99487939 170.04172886131559 12.99488571238917 170.04188588 12.99441055 170.08698664 13.00556577 170.08698664 13.00556577,mast:HST/product/idxr22060_drz.jpg,mast:HST/product/idxr22060_drz.fits,PUBLIC,False,nan,26250740,770402919,770402919,28.036980630951845
science,HST,HAP-SVM,WFC3/UVIS,HAP,F814W,OPTICAL,SN-2009HD,EXT-STAR;SUPERNOVA TYPE II,hst_12968_01_wfc3_uvis_f814w_ic0h01,170.0707916667,12.97952777778,image,"Maund, Justyn Robert",3,56595.03795848,56595.06025013,1668.0,702.5,961.2,--,56960.19879628,12968,HAP,--,POLYGON 170.06672226868847 12.972905586295301 170.0635257891145 12.983812019710173 170.0749104903491 12.98612326584507 170.07801614877064 12.975216695838752 170.06672226868847 12.972905586295301,mast:HST/product/hst_12968_01_wfc3_uvis_f814w_ic0h01_drc.jpg,mast:HST/product/hst_12968_01_wfc3_uvis_f814w_ic0h01_drc.fits,PUBLIC,False,nan,26583220,656236841,656236841,27.75725548690898
science,HST,HAP-SVM,WFC3/UVIS,HAP,F814W,OPTICAL,NGC3627,GALAXY;SPIRAL,hst_13364_12_wfc3_uvis_f814w_icdm12,170.06225,12.96494722222,image,"Calzetti, Daniela",3,56696.73943426,56696.75373961,980.0,702.5,961.2,--,56697.09958337,13364,HAP,--,POLYGON 170.09587933272053 12.960105704016119 170.05863460197395 12.933661530475247 170.05809257526263 12.93424481735564 170.05776510697393 12.934156767914493 170.04302769524398 12.950961599666966 170.04318577063316 12.951280772110136 170.04299377906062 12.951456847177832 170.0426775854856 12.951379785249223 170.02768964516855 12.968425858592644 170.06500406874247 12.995633812326867 170.06530902619218 12.995380680884194 170.06580600422274 12.995721846164919 170.0807816559278 12.978277394498823 170.080634809219 12.977958243932939 170.08081550140184 12.97779314765865 170.0811091531106 12.977881170777074 170.09583423770147 12.960677998656557 170.095642216588 12.960413890106555 170.09587933272053 12.960105704016119,mast:HST/product/hst_13364_12_wfc3_uvis_f814w_icdm12_drc.jpg,mast:HST/product/hst_13364_12_wfc3_uvis_f814w_icdm12_drc.fits,PUBLIC,False,nan,26606940,653027433,653027433,0.0
science,HST,CALWF3,WFC3/UVIS,HST,F275W,UV,NGC3627,GALAXY;SPIRAL,icdm12030,170.06225,12.96494722222,image,"Calzetti, Daniela",3,56696.5926053588,56696.62305667824,2361.0,243.20000000000002,305.8,LEGUS: Legacy ExtraGalactic UV Survey,56696.8609953,13364,GO,--,POLYGON 170.09580969 12.960652099999949 170.08097361000009 12.977782859999929 170.08045708738919 12.977413690610373 170.08042303021929 12.977453011541211 170.08079944000002 12.977725710000051 170.08050660602714 12.978063037290891 170.08076328 12.97824899000004 170.06572730999986 12.995567879999875 170.06521380355662 12.995195839564763 170.06495993000001 12.995488229999875 170.02762464999995 12.96843275999999 170.04266164000018 12.951116050000078 170.04291838622683 12.951302132547198 170.04305495352725 12.951144845321066 170.04290102000007 12.

In [9]:
%%time
cd(hstdir)
print("Downloading data.\nThis may take several minutes... \n")
products = Observations.get_product_list(obs_table)
data_prod = ['FLC']       
data_type = ['HAP-SVM']   

Observations.download_products(products, project=data_type, 
                               productSubGroupDescription=data_prod)
print("Done")


This may take several minutes... 

INFO: 52 of 104 products were duplicates. Only returning 52 unique product(s). [astroquery.mast.utils]
Done
CPU times: user 20.9 s, sys: 17.5 s, total: 38.3 s
Wall time: 4min 6s


In [11]:
# Gathering the FLC file names
input_flcs = glob.glob(os.path.join('mastDownload','HST','*','*.fits'))

# Moving files from mastDownload directory into current directory
for flc in input_flcs:
  shutil.copy(flc, os.path.basename(flc))

shutil.rmtree('mastDownload') # removes the extra directories
print("Done")

Done


In [13]:
flc_files = glob.glob("hst*_acs_wfc_*_j*_flc.fits")
flc_files.sort()       # Alphabetizing files

for i,f in enumerate(flc_files):
    try: hdu = fits.open(f)
    except: print(f,"not good. Please remove.")

## F555W
The following code will be run in a python terminal

In [ ]:
from drizzlepac.astrodrizzle import AstroDrizzle as adriz

flc_files = glob.glob("*f555w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f555w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False)  

# F814W

In [ ]:
flc_files = glob.glob("*f814w*flc*.fits")
flc_files.sorqqt()

adriz(flc_files, output='M66_mosaic_uvis_f814w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

# F438W

In [ ]:
flc_files = glob.glob("*f438w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f438w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

# F336W

In [ ]:
flc_files = glob.glob("*f336w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f336w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

# F275W

In [ ]:
flc_files = glob.glob("*f275w*flc*.fits")
flc_files.sort()

adriz(flc_files, output='M66_mosaic_uvis_f275w',
      preserve=True, clean=True, context=False,
      skymethod='globalmin+match', # renormalizes the brightness of each field
      driz_cr=True, driz_cr_corr=False,
      final_wcs=True, final_rot=0,  # places North pointed up
      configobj=None, overwrite=False, build=False) 

# Photometry

## F814W

In [ ]:
f814w_data = fits.getdata(f814w)
f814w_data_sliced = f814w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
f814w_objects = DaoFindObjects(f814w_data_sliced, fwhm=0.10*2, pixtoarcs=pixtoarcs, threshold=2, plot=True)

# investigating the daofind sources and extracting their photometry
f814w_hdu = fits.open(f814w)
apcorr, aperr, apcorr_ext, aperr_ext = RunPhots(f814w_hdu, gal="M66", instrument="WFC3",
                                               filter='F814W', fwhm_arcs=0.10*2, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f814w_hdu.close()

# ((np.float64(-0.8588325849729528),),
#  np.float64(0.1807607521654574),
#  np.float64(-0.3719457463759386),
#  np.float64(0.10642947110641915))

## F555W

In [ ]:
f555w_data = fits.getdata(f555w)
f555w_data_sliced = f555w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
fwhm = 0.18 * 2 # originally found 0.18 arcsec through ds9
f555w_objects = DaoFindObjects(f555w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True)

#investigating the daofind sources and extracting their photometry
f555w_hdu = fits.open(f555w)
apcorr555w, aperr555w, apcorr_ext555w, aperr_ext555w = RunPhots(f555w_hdu, gal="M66", instrument="WFC3",
                                               filter='F555W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f555w_hdu.close()

# ((np.float64(-0.8814290765714752),),
#  np.float64(0.24055442323969883),
#  np.float64(-0.3472747023393392),
#  np.float64(0.12601848622183956))

## F438W

In [ ]:
f438w_data = fits.getdata(f438w)
f438w_data_sliced = f438w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
fwhm = 0.11 * 2 # originally found 0.18 arcsec through ds9
f438w_objects = DaoFindObjects(f438w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True, vmax=0.09)
print(len(f438w_objects)) # found with 195 sources fwhm=0.10

#investigating the daofind sources and extracting their photometry
f438w_hdu = fits.open(f438w)
apcorr438w, aperr438w, apcorr_ext438w, aperr_ext438w = RunPhots(f438w_hdu, gal="M66", instrument="WFC3",
                                               filter='F438W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f438w_hdu.close()

# ((np.float64(-0.9175731144364969),),
#  np.float64(0.2291917542835823),
#  np.float64(-0.4745254777532202),
#  np.float64(0.15543813531799572))

## F336W

In [ ]:
f336w_data = fits.getdata(f336w)
f336w_data_sliced = f336w_data[3400:3700, 4400:4700]
pixtoarcs = 0.03962
fwhm = 0.12 * 2 # originally found 0.18 arcsec through ds9
f336w_objects = DaoFindObjects(f336w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True, vmax=0.05)
print(len(f336w_objects)) # found with 195 sources fwhm=0.10

# investigating the daofind sources and extracting their photometry
f336w_hdu = fits.open(f336w)
apcorr336w, aperr336w, apcorr_ext336w, aperr_ext336w = RunPhots(f336w_hdu, gal="M66", instrument="WFC3",
                                               filter='F336W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f336w_hdu.close()

# ((np.float64(-0.9540754478692428),),
#  np.float64(0.23448030394882666),
#  np.float64(-0.43512525755218445),
#  np.float64(0.13649683194194648))

## F275W

In [ ]:
f275w_data = fits.getdata(f275w)
f275w_data_sliced = f275w_data[4400:5000, 4300:4900]
pixtoarcs = 0.03962
fwhm = 0.12 * 2 # originally found 0.18 arcsec through ds9
f275w_objects = DaoFindObjects(f275w_data_sliced, fwhm=fwhm, pixtoarcs=pixtoarcs,
                               threshold=2, plot=True, vmax=0.015)
print(len(f275w_objects)) # found with 195 sources fwhm=0.10

#investigating the daofind sources and extracting their photometry
f275w_hdu = fits.open(f275w)
apcorr275w, aperr275w, apcorr_ext275w, aperr_ext275w = RunPhots(f275w_hdu, gal="M66", instrument="WFC3",
                                               filter='F275W', fwhm_arcs=fwhm, threshold=2,
                                               num_stars=20, extended_rad=8, reg_correction=[1,1])
f275w_hdu.close()

# ((np.float64(-0.9707309801124901),),
#  np.float64(0.25887846989477437),
#  np.float64(-0.4419304737417187),
#  np.float64(0.1511766174865048))

# Astrometric Corrections

M66_CSC = LoadSources(chandra_hst_dir+"cscresults_M66_renamed.frame")
M66_unique = FindUnique(M66_CSC, header="CSC ID")
WriteReg(M66_unique, outfile=chandra_hst_dir+"M66_cscsources.reg", idheader="CSC ID", 
         radius=5, radunit="arcsec", width=2, color="hotpink")

In [ ]:
M66_calibrators = ["2CXO J112014.8+125814",
                   "2CXO J112018.9+130135",
                   "2CXO J112019.1+125947", # Not sure about this (upper left corner)
                   "2CXO J112019.5+125958", # Not sure about this (upper left corner)
                   "2CXO J112018.8+125924",
                  ] 
print(f"{len(M66_calibrators)} calibrators to match")

M66_calibrators = RemoveElse(M66_unique, keep=M66_calibrators, header='CSC ID')
print(f"{len(M66_calibrators)} calibrators found")

WriteReg(M66_calibrators, outfile=chandra_hst_dir+"M66_calibrators.reg")

In [ ]:
base_coords = GetCoords(hstdir+"M66_daofind_f555w_wfc3_fk5.reg")
cat_coords = [M66_calibrators["RA"].values.tolist(),
              M66_calibrators["Dec"].values.tolist()]

CorrectAstrometry(base_coords, cat_coords, returnshifts=True,
                 savebasereg=chandra_hst_dir+"M66_astrocorrect.reg")

# Input min and max separation: 0, 9.6e-5
# Min and max x residuals: -3e-5, 2.2e-5
# Min and max y residuals: -4.2e-5, 1.6e-5

# RESULTS OF ASTROMETRY CORRECTION

# Median offsets in x and y:
#  -2.018614964072185e-05 3.766607498967289e-05 degrees
#  -0.07267013870659866 0.13559786996282241 arcsecs

# Standard deviation in x and y offsets:
#  1.7643739978865733e-05 1.9961232387211725e-05 degrees
#  0.06351746392391663 0.07186043659396221 arcsecs

#  (np.float64(-2.018614964072185e-05),
#  np.float64(3.766607498967289e-05),
#  np.float64(1.7643739978865733e-05),
#  np.float64(1.9961232387211725e-05))

Assuming that the above astrometric corrections and its outputs are correct, since the standard deviations in the x and y offsets are bigger than their corresponding median offsets, there is not any correction needed between them. 

## Calculating Positional Uncertainties

In [ ]:
sig1, sig2 = CalcPU(M66_CSC, std=[0.06352,0.07186], sig2search='Err Ellipse Major')

In [ ]:
M66_CSC['1Sig'] = sig1  # Saves sig1 to a new header called '1Sig'
M66_CSC['2Sig'] = sig2  # Saves sig2 to a new header called '2Sig'

# The new DataFrame has 1 and 2sig added at the end of the columns list
display(M66_CSC)

M66_CSC.to_csv(chandra_hst_dir+'M66_csc_astrocorrected.frame')

In [ ]:
# Pulling the ID of each unique CSC source
ids = FindUnique(M66_CSC, header="CSC ID")["CSC ID"].values.tolist()

# Building an empty DataFrame, which I will fill below
M66_best = BuildFrame()

for i in ids: # for each unique ID pulled from CSC...
    
    # Search for all instances (i.e. observations) of each source
    Temp = Find(M66_CSC, "CSC ID = " + i)
    
    # Try to avoid sources where counts = NaN (invalid observations) 
    if len(Find(Temp, ["Counts != NaN", "Theta != NaN"])) > 0: 
        Temp = Find(Temp, ["Counts != NaN", "Theta != NaN"])
    
    # Specifically focus on those with a valid number of counts
    if len(Find(Temp, "Counts > 0")) > 0: 
        Temp = Find(Temp, ["Counts > 0"])
        
    # Otherwise, all instances with counts = 0 have the same measurements, 
    # so it doesn't matter which row is chosen for the best radii
    else: pass;
    
    # Take the source with the smallest 2sig. 
    # If there's more than one, take the first on the list.
    Tempbest = Find(Temp, "2Sig =< " + str(min(Temp["2Sig"]))).iloc[:1]
    
    # Add the chosen source observation to the new DataFrame
    M66_best = pd.concat([M66_best, Tempbest], ignore_index=True)

# Saving results to a DataFrame file
# This file contains only rows from M101_CSC that has the 
# best 2sigma radius, based on the search performed above
M66_best.to_csv(chandra_hst_dir+"M66_csc_bestrads.frame")

display(M66_best)

In [ ]:
# Saving the positional uncertainties as region files
WriteReg(M66_best, radius=M66_best['2Sig'].values.tolist(), radunit='arcsec', \
         idheader="CSC ID", outfile=chandra_hst_dir+'M66_bestrads_2sig.reg')
WriteReg(M66_best, radius=M66_best['1Sig'].values.tolist(), radunit='arcsec', \
         outfile=chandra_hst_dir+'M66_bestrads_1sig.reg')